<h1>DETECÇÃO DE BORDAS COM O ALGORITMO DE CANNY</h1>

<h2>Sumário</h2>
    
[1. Introdução](#introducao)<br>
[2. Script melhorado da técnica o Pontilhismo](#script1)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.1 Bibliotecas](#bibliotecas)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.2 Função de aplicação da técnica do Pontilhismo](#funcao1)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.3 Resultados](#result)<br>

<a id='introducao'></a>

<h2>1. Introdução</h2>

<p style='text-align: justify;'>Este exercício tem com objetivo o entendimento acerca do algoritmo de Canny para a detecção de bordas em uma figura. Além disso, há um viés artístico no aprendizado e implementação da técnica do pontilhismo. Dessa forma, pretende-se implementar uma melhoria no algoritmo padrão do pontilhismo através da utilização das informações referentes às bordas da figura fornecidas pelo algoritmo de Canny.</p>

<a id='script1'></a>

<h2>2. Script melhorado da técnica do Pontilhismo</h2>

<p style='text-align: justify;'>Esta seção descreverá a implementação do algoritmo descrito na Introdução.</p>

<a id='bibliotecas'></a>

<h3>2.1 Bibliotecas</h3>

<p style='text-align: justify;'>Inicialmente, importa-se as bibliotecas necessárias para a realização dos algoritmos. A primeira consiste na biblioteca do openCV para Python, utilizada para todo o tratamento de imagem. O numpy é utilizado para trabalhar com as matrizes das imagens. A biblioteca random foi aplicada no posicionamento aleatório dos círculos para o Pontilhismo.</p>

In [1]:
import cv2
import numpy as np
import random

<a id='funcao1'></a>

<h3>2.2 Função de aplicação da técnica do Pontilhismo</h3>

A função <i>pontilhismo</i> exibida abaixo recebe como parâmetro a variável STEP, que indicará a taxa em que a imagem original será amostrada para o posicionamento das circunferências. O parâmetro JITTER indica o valor máximo de um deslocamento aleatório do ponto de posicionamento das circunferências. O RAIO indica o raio das circunferências que serão geradas. Além disso, a imagem original é passada como parâmetro.

Dentro da função, define-se os vetores de coordenadas x e y de acordo com o STEP definido. Em seguida, é definida a martriz <i>points</i> que será a imagem de saída. Esta matriz é criada com a cor completamente branca.

Dentro dos loops, os vetores de coordenadas são acessados de forma aleatória e uma circunferência é posicionada num ponto aleatório em torno das coordenadas de centro definidas. Dessa forma, tem-se uma imagem formada apenas por circunferências espalhadas.

In [2]:
def pontilhismo(STEP, JITTER, RAIO, image):
    width = image.shape[1]
    height = image.shape[0]
    
    xrange = np.arange(0, height//STEP, 1, dtype='int')
    yrange = np.arange(0, width//STEP, 1, dtype='int')
    
    for i in range (len(xrange)):
        xrange[i] = xrange[i]*STEP+STEP/2
    
    for i in range (len(yrange)):
        yrange[i] = yrange[i]*STEP+STEP/2
     
    points = np.full(image.shape, 255, dtype='uint8')
    random.shuffle(xrange)
    
    for i in xrange:
        random.shuffle(yrange)
        for j in yrange:
            x = i + random.randint(1-JITTER,JITTER)
            y = j + random.randint(1-JITTER,JITTER)
            #print("x: "+str(x)+" y: ",str(y), image.shape )
            if x < image.shape[0] and y < image.shape[1]:
                gray = int(image[x][y])
                cv2.circle(points, (y,x), RAIO, (gray,gray,gray), -1 )
    return points

O trecho de código abaixo mostra o processo de abertura da imagem alvo do processo e a verificação da abertura:

In [3]:
image = cv2.imread("..\images\\we.jpg",0)

if not image.data:
    print("Erro ao abrir imagem")

Abaixo, tem-se os parâmetros da função do pontilhismo e do algoritmo de Canny. Os parâmetros STEP e JITTER já foram anteriormente explicados. A variável STEP_CANNY indica o passo em que os limiares do algoritmo de Canny irão avançar, tendo como limite a variável CANNY_MAX.

Dado que uma das propostas do algoritmo é a variação dos limiares de do algoritmo de Canny e eventual preenchimento das bordas com circunferências de raios diferente para cada variação, criou-se uma variável RAIO_MAX, a qual indica qual será o raio da maior circunferência que será utilizada nas bordas. O menor raio considerado foi de 1, o qual não está definido em variáveis. O RAIO_INICIAL indica o raio das circunferências que inicialmente são utilizadas para o preenchimento da imagem.

In [25]:
STEP = 2
JITTER = 5
STEP_CANNY = 20
CANNY_MAX = 220
RAIO_MAX = 3
RAIO_INICIAL = 6

Com as informações anteriores, criou-se um vetor RAIO de inteiros e com o mesmo tamanho da quantidade de passos nos limiares de Canny. Como a quantidade de passos depende se CANNY_MAX é múltiplo de STEP_CANNY, utilizou-se uma condição para que o tamanho do vetor fosse variável obedecendo a isto. Como os raios das circunferências são inteiros, o tipo do vetor foi declarado como 'uint'. Além disso, o raio deverá tender a diminuir com o passar das iterações, então os valores partem do RAIO_MAX para o raio unitário.

In [26]:
RAIO = np.linspace(RAIO_MAX, 1, CANNY_MAX//STEP_CANNY - 1 if CANNY_MAX%50==0 else CANNY_MAX//STEP_CANNY, dtype='uint')
print(RAIO)

[3 2 2 2 2 2 1 1 1 1 1]


Abaixo tem-se a execução da função do Pontilhismo, preenchendo a imagem com circunferências de raio RAIO_INICIAL:

In [31]:
pontos = pontilhismo(STEP, JITTER, RAIO_INICIAL, image)
cv2.imshow("image",pontos)
cv2.waitKey(0)
cv2.destroyAllWindows()

Para a melhoria na imagem criada inicialmente acima, foi criado um loop externo, o qual dita os valores dos limiares de Canny baseado nos parâmetros anteriormente descritos. O parâmetro superior é definido como sendo o triplo do inferior. A matriz <i>bordas</i> gerada possui fundo preto, tendo como cor branca apenas as bordas das figuras. Dessa forma, a função <i>np.where</i> possibilita que a variável <i>pontos_borda</i> possua as coordenadas de todos os pontos de borda detecdados.

Com estes pontos de interesse, realiza-se outro loop de forma a acessar as coordenadas das bordas e preenchê-las com circunferências de radio variável dependendo da iteração. Para garantir esta variação dos raios, a variável <i>inc</i> foi criada para ser incrementada e acessar os outros valores de raios no vetor RAIO.

In [32]:
inc = 0
for c in range(STEP_CANNY,CANNY_MAX,STEP_CANNY):
    bordas = cv2.Canny(image, c, 3*c)
    pontos_borda = np.where(bordas == 255)
    for i in range(len(pontos_borda[0])):
        x = pontos_borda[0][i]
        y = pontos_borda[1][i]
        gray = int(image[x][y])
        cv2.circle(pontos, (y,x), RAIO[inc], (gray,gray,gray), -1)
    inc += 1

Abaixo tem-se as linhas de código referentes à exibição do resultado:

In [34]:
cv2.imshow("pontos",pontos)
cv2.waitKey(0)
cv2.destroyAllWindows()

<a id='result'></a>

<h3>2.3 Resultados</h3>

A imagem original utilizada para o processo anteriormente descrito foi a seguinte:

![original](../images/we.jpg)

Após o primeiro processo de preenchimento da imagem com circunferências, obteve-se o seguinte:

![pont1](../images/documentation/pont1.jpg)

Percebe-se que apenas com o processo comum de Pontilhismo, o resultado não fica visualmente agradável. Porém, ao utilizar as informações de borda das imagens obtidas com o algoritmo de Canny para melhora o processo, obteve-se o seguinte resultado:

![pont_final](../images/documentation/pont_final.jpg)

É notável que a imagem final fica visualmente mais agradável se comparada com aquela gerada no processo intermediário.